In [2]:
import json
import pandas as pd

df = pd.read_csv('all_reviews_with_raw_1star.csv')


In [3]:
df

,_row_index,title,totalScore,google_place_id,reviewsCount,name,reviewerNumberOfReviews,isLocalGuide,textTranslated,publishAt,publishedAtDate,visitedIn,likesCount,stars,raw_output
0,12,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Sarah C,29,True,I would like to share my experience after a ye...,6 months ago,2025-02-24T08:14:41.537Z,Visited in February,0,1,model='gemma3:1b' created_at='2025-09-15T13:37...
1,43,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Anaïs P. (Niassah Siana),20,True,"€1,500 for a promotional offer for 8 sessions ...",Edited 10 months ago,2024-10-29T10:33:57.272Z,Visited in October 2023,0,1,model='gemma3:1b' created_at='2025-09-15T13:37...
2,55,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Fabienne Oury,5,False,"Very disappointed, I made an appointment on do...",a year ago,2024-08-05T13:44:17.512Z,Visited in August 2024,0,1,model='gemma3:1b' created_at='2025-09-15T13:37...
3,125,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Inès Léaument,2,False,"Disappointed with the results, the agency's on...",a year ago,2024-06-26T19:41:05.830Z,Visited in June 2024,0,1,model='gemma3:1b' created_at='2025-09-15T13:37...
4,126,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Laurine Jlv,4,False,Friendly staff but after 8 hair removal sessio...,a year ago,2024-06-26T19:40:19.475Z,Visited in July 2023,0,1,model='gemma3:1b' created_at='2025-09-15T13:38...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823,29260,Lazeo La Defense,4.7,ChIJx6pR16xl5kcRS4HSC_KGX7I,395,Ryane Belaid,32,True,At the beginning it's good then no customer fo...,3 years ago,2022-06-17T10:31:02.498Z,Visited in June 2022,0,1,model='gemma3:1b' created_at='2025-09-15T20:29...
1824,29264,Lazeo La Defense,4.7,ChIJx6pR16xl5kcRS4HSC_KGX7I,395,Derome Adélaïde,9,False,NaN,3 years ago,2022-05-28T06:01:41.937Z,Visited in May 2022,0,1,model='gemma3:1b' created_at='2025-09-15T20:29...
1825,29269,Lazeo La Defense,4.7,ChIJx6pR16xl5kcRS4HSC_KGX7I,395,Elodie TORDEUX,13,False,Absolutely avoid!!!\nI made a first informatio...,3 years ago,2022-04-21T13:15:53.963Z,Visited in April 2022,0,1,model='gemma3:1b' created_at='2025-09-15T20:29...
1826,29281,Lazeo La Defense,4.7,ChIJx6pR16xl5kcRS4HSC_KGX7I,395,Ouarda Fali,4,False,I had a laser session today because my previou...,3 years ago,2021-12-15T22:43:27.439Z,Visited in December 2021,0,1,model='gemma3:1b' created_at='2025-09-15T20:29...


In [9]:
import re
import json
import ast
import pandas as pd
from typing import Optional, List, Dict, Any

INPUT_CSV = "all_reviews_with_raw_2star.csv"
OUTPUT_CSV = "all_reviews_with_issues_2star.csv"
FAILED_SAMPLE_CSV = "failed_rows_sample.csv"
SAMPLE_FAILS = 100  # how many failed samples to save

# ---------------- helpers ----------------
def safe_json_loads(txt: str) -> Optional[Dict]:
    """Try multiple ways to parse a JSON-like string into a dict."""
    if txt is None:
        return None
    s = str(txt).strip()
    if not s:
        return None
    # ensure we focus on the outer {...}
    if not (s.startswith("{") and s.endswith("}")):
        first = s.find("{")
        last = s.rfind("}")
        if first != -1 and last != -1 and last > first:
            s = s[first:last+1]

    # 1) normal json
    try:
        return json.loads(s)
    except Exception:
        pass
    # 2) ast.literal_eval (handles single quotes and Python literals)
    try:
        return ast.literal_eval(s)
    except Exception:
        pass
    # 3) remove trailing commas
    try:
        cleaned = re.sub(r",\s*([}\]])", r"\1", s)
        return json.loads(cleaned)
    except Exception:
        pass
    # 4) naive swap unescaped single quotes to double quotes then parse
    try:
        swapped = re.sub(r"(?<!\\)'", '"', s)
        return json.loads(swapped)
    except Exception:
        pass
    # 5) try to normalize whitespace and unescape common sequences
    try:
        t = s.replace("\\n", "\n").replace("\\t", "\t")
        t = re.sub(r"\s+", " ", t)
        t = re.sub(r"(?<!\\)'", '"', t)
        t = re.sub(r",\s*}", "}", t)
        t = re.sub(r",\s*\]", "]", t)
        return json.loads(t)
    except Exception:
        return None

def unescape_string_literal(s: str) -> str:
    """Unescape common escaped sequences from a quoted CSV field (best-effort)."""
    if s is None:
        return ""
    s = str(s)
    # strip surrounding quotes if present
    if (s.startswith("'") and s.endswith("'")) or (s.startswith('"') and s.endswith('"')):
        s = s[1:-1]
    try:
        # convert sequences like \n, \" etc.
        return bytes(s, "utf-8").decode("unicode_escape")
    except Exception:
        return s

def extract_content_from_Message(cell_text: str) -> Optional[str]:
    """
    Try to extract the content=... inside Message(... content='...') or similar patterns and unescape it.
    Returns the unescaped content string or None.
    """
    if not cell_text:
        return None
    # capture inside Message(...)
    m = re.search(r"Message\s*\((.*)\)\s*$", cell_text, flags=re.DOTALL)
    inside = m.group(1) if m else cell_text
    # find content='...' or content="..."
    mc = re.search(r"content\s*=\s*(?:'((?:\\'|[^'])*)'|\"((?:\\\"|[^\"])*)\")", inside, flags=re.DOTALL)
    if mc:
        raw = mc.group(1) if mc.group(1) is not None else mc.group(2)
        return unescape_string_literal(raw)
    # fallback: maybe message= contains backticks or raw json; try triple-backtick inside cell
    m2 = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", cell_text, flags=re.DOTALL)
    if m2:
        return m2.group(1)
    # otherwise return None
    return None

def find_json_candidates(cell_text: str) -> List[str]:
    """
    Return a list of candidate JSON-like blocks (strings) found in the cell text.
    Tries multiple heuristics: triple-backticks, Message.content, large {...} fragments.
    """
    if not cell_text:
        return []
    txt = str(cell_text)

    blocks = []
    # 1) explicit triple-backtick json blocks
    for m in re.finditer(r"```json\s*(\{.*?\})\s*```", txt, flags=re.DOTALL):
        blocks.append(m.group(1))

    # 2) generic triple-backtick blocks
    if not blocks:
        for m in re.finditer(r"```(?:json)?\s*(.*?)\s*```", txt, flags=re.DOTALL):
            cand = m.group(1).strip()
            if cand.startswith("{") and cand.endswith("}"):
                blocks.append(cand)

    # 3) try extracting content= from Message(...) (unescaped)
    if not blocks:
        content = extract_content_from_Message(txt)
        if content:
            # content may itself contain triple-backticks; try to extract the braces inside
            m_back = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", content, flags=re.DOTALL)
            if m_back:
                blocks.append(m_back.group(1))
            else:
                # extract the first { ... } inside content
                first = content.find("{")
                last = content.rfind("}")
                if first != -1 and last != -1 and last > first:
                    blocks.append(content[first:last+1])
                else:
                    blocks.append(content)

    # 4) fallback: any sizeable {...} fragment from the whole text
    if not blocks:
        for m in re.finditer(r"(\{(?:[^{}]|\{[^{}]*\}){30,}\})", txt, flags=re.DOTALL):
            blocks.append(m.group(1))

    return blocks

def extract_metadata(cell_text: str) -> Dict[str, Any]:
    meta = {}
    if not cell_text:
        return meta
    mm = re.search(r"model=(?:'([^']*)'|\"([^\"]*)\"|(\S+))", cell_text)
    if mm:
        meta['model'] = mm.group(1) or mm.group(2) or mm.group(3)
    ca = re.search(r"created_at=(?:'([^']*)'|\"([^\"]*)\"|(\S+))", cell_text)
    if ca:
        meta['created_at'] = ca.group(1) or ca.group(2) or ca.group(3)
    dr = re.search(r"done_reason=(?:'([^']*)'|\"([^\"]*)\"|(\S+))", cell_text)
    if dr:
        meta['done_reason'] = dr.group(1) or dr.group(2) or dr.group(3)
    return meta

# ---------------- main ----------------
def run_extract_and_merge(input_csv=INPUT_CSV,
                          output_csv=OUTPUT_CSV,
                          failed_csv=FAILED_SAMPLE_CSV,
                          sample_fails=SAMPLE_FAILS,
                          raw_col="raw_output"):
    df = pd.read_csv(input_csv, dtype=str).fillna("")
    if raw_col not in df.columns:
        raise ValueError(f"CSV must contain a '{raw_col}' column. Found: {df.columns.tolist()}")

    extracted = []
    failed_examples = []
    stats = {
        "rows_with_message": 0,
        "rows_with_backtick": 0,
        "rows_with_content_token": 0,
        "rows_with_candidates": 0,
        "total_extracted_issues": 0
    }

    for src_idx, cell in df[raw_col].items():
        text = str(cell)
        if "Message(" in text or "message=" in text:
            stats["rows_with_message"] += 1
        if "```json" in text or "```" in text:
            stats["rows_with_backtick"] += 1
        if "content=" in text:
            stats["rows_with_content_token"] += 1

        candidates = find_json_candidates(text)
        if candidates:
            stats["rows_with_candidates"] += 1

        parsed_any = False
        for blk_idx, cand in enumerate(candidates):
            # if cand looks escaped (contains \n or \" ), unescape first and try
            parsed = safe_json_loads(cand)
            if not parsed and ('\\n' in cand or '\\"' in cand or "\\'" in cand):
                cand_unesc = unescape_string_literal(cand)
                parsed = safe_json_loads(cand_unesc)

            if parsed and isinstance(parsed, dict) and "issues" in parsed and isinstance(parsed["issues"], list):
                parsed_any = True
                for issue_obj in parsed["issues"]:
                    row = extract_metadata(text)
                    # flatten issue object
                    if isinstance(issue_obj, dict):
                        for k, v in issue_obj.items():
                            row[k] = v
                    else:
                        row["issue"] = issue_obj
                    row["source_row"] = src_idx
                    row["json_block_idx"] = blk_idx
                    extracted.append(row)
                    stats["total_extracted_issues"] += 1
            elif parsed:
                # parsed but no issues key: store parsed top-level fields
                parsed_any = True
                row = extract_metadata(text)
                for k, v in parsed.items():
                    if isinstance(v, (str, int, float, bool)) or v is None:
                        row[k] = v
                    else:
                        row[k] = json.dumps(v, ensure_ascii=False)
                row["source_row"] = src_idx
                row["json_block_idx"] = blk_idx
                row["_parsed_no_issues"] = True
                extracted.append(row)

        if not parsed_any:
            if len(failed_examples) < sample_fails:
                failed_examples.append({
                    "source_row": src_idx,
                    "raw_preview": text[:2000].replace("\n","\\n"),
                    "candidates_found": candidates[:3]
                })

    # build extracted DataFrame
    if extracted:
        issues_df = pd.DataFrame(extracted)
    else:
        issues_df = pd.DataFrame(columns=["source_row", "issue", "actionable"])

    # Merge back to original df (left join keeps all original rows)
    merged = df.reset_index(drop=False).rename(columns={"index":"orig_index"})
    # orig_index equals original integer index; we'll use that to merge with source_row
    # ensure source_row in issues_df is int
    if "source_row" in issues_df.columns:
        issues_df['source_row'] = issues_df['source_row'].astype(int)
    merged = merged.merge(issues_df, left_on="orig_index", right_on="source_row", how="left")

    # Save outputs
    merged.to_csv(output_csv, index=False)
    # Save failed samples for inspection
    pd.DataFrame(failed_examples).to_csv(failed_csv, index=False)

    # Print diagnostics
    print("=== Diagnostics ===")
    print(f"Total source rows: {len(df)}")
    print(f"Rows with Message token: {stats['rows_with_message']}")
    print(f"Rows with backtick token: {stats['rows_with_backtick']}")
    print(f"Rows with content= token: {stats['rows_with_content_token']}")
    print(f"Rows with any candidates found: {stats['rows_with_candidates']}")
    print(f"Total extracted issue rows: {stats['total_extracted_issues']}")
    print("===================")
    print(f"Merged saved to: {output_csv}")
    print(f"Failed sample rows saved to: {failed_csv}")
    return merged, issues_df, failed_examples

# ---------------- run ----------------
if __name__ == "__main__":
    merged_df, issues_df, failed = run_extract_and_merge()
    # quick preview
    print("\nMerged preview (first 10 rows):")
    print(merged_df.head(10).to_string(index=False))
    print("\nExtracted issues preview (first 10 rows):")
    print(issues_df.head(10).to_string(index=False))
    if failed:
        print(f"\nSaved {len(failed)} failed samples to inspect. Check '{FAILED_SAMPLE_CSV}'.")


=== Diagnostics ===
Total source rows: 330
Rows with Message token: 330
Rows with backtick token: 330
Rows with content= token: 330
Rows with any candidates found: 330
Total extracted issue rows: 1343
Merged saved to: all_reviews_with_issues_2star.csv
Failed sample rows saved to: failed_rows_sample.csv

Merged preview (first 10 rows):
 orig_index _row_index              title totalScore             google_place_id reviewsCount           name reviewerNumberOfReviews isLocalGuide                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

## merged_df

In [47]:
pd.set_option("display.max_rows", 10)

In [4]:
merged_df.issue.unique()

NameError: name 'merged_df' is not defined

In [8]:
merged_df1 = merged_df.copy()

In [10]:
merged_df2 = merged_df.copy()

In [11]:
merged_df1.orig_index.nunique()

1828

In [12]:
merged_df2.orig_index.nunique()

330

In [13]:
final_merged_df = pd.concat([merged_df1,merged_df2])

In [14]:
final_merged_df

,orig_index,_row_index,title,totalScore,google_place_id,reviewsCount,name,reviewerNumberOfReviews,isLocalGuide,textTranslated,...,raw_output,model,created_at,done_reason,issue,actionable,source_row,json_block_idx,priority,quality_check
0,0,12,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Sarah C,29,True,I would like to share my experience after a ye...,...,model='gemma3:1b' created_at='2025-09-16T16:40...,gemma3:1b,2025-09-16T16:40:16.3469944Z,stop,hair regrowth complications,investigate machine programming,0.0,0.0,NaN,NaN
1,0,12,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Sarah C,29,True,I would like to share my experience after a ye...,...,model='gemma3:1b' created_at='2025-09-16T16:40...,gemma3:1b,2025-09-16T16:40:16.3469944Z,stop,poor treatment results,consult specialist,0.0,0.0,NaN,NaN
2,0,12,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Sarah C,29,True,I would like to share my experience after a ye...,...,model='gemma3:1b' created_at='2025-09-16T16:40...,gemma3:1b,2025-09-16T16:40:16.3469944Z,stop,lack of compensation for ineffective sessions,escalate complaint,0.0,0.0,NaN,NaN
3,0,12,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Sarah C,29,True,I would like to share my experience after a ye...,...,model='gemma3:1b' created_at='2025-09-16T16:40...,gemma3:1b,2025-09-16T16:40:16.3469944Z,stop,lack of doctor consultation,request consultation,0.0,0.0,NaN,NaN
4,0,12,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Sarah C,29,True,I would like to share my experience after a ye...,...,model='gemma3:1b' created_at='2025-09-16T16:40...,gemma3:1b,2025-09-16T16:40:16.3469944Z,stop,lack of clear diagnosis and treatment plan,request detailed explanation,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,329,29223,Lazeo La Defense,4.7,ChIJx6pR16xl5kcRS4HSC_KGX7I,395,Eloïse N,14,True,I don't recommend this center. I had my last t...,...,model='gemma3:1b' created_at='2025-09-16T16:36...,gemma3:1b,2025-09-16T16:36:43.5584584Z,stop,unclean glasses,inspect glasses and ensure disinfection,329.0,0.0,NaN,NaN
1342,329,29223,Lazeo La Defense,4.7,ChIJx6pR16xl5kcRS4HSC_KGX7I,395,Eloïse N,14,True,I don't recommend this center. I had my last t...,...,model='gemma3:1b' created_at='2025-09-16T16:36...,gemma3:1b,2025-09-16T16:36:43.5584584Z,stop,rushed sessions,review session duration and ensure adequate ti...,329.0,0.0,NaN,NaN
1343,329,29223,Lazeo La Defense,4.7,ChIJx6pR16xl5kcRS4HSC_KGX7I,395,Eloïse N,14,True,I don't recommend this center. I had my last t...,...,model='gemma3:1b' created_at='2025-09-16T16:36...,gemma3:1b,2025-09-16T16:36:43.5584584Z,stop,inconsistent results,request detailed session notes and results ana...,329.0,0.0,NaN,NaN
1344,329,29223,Lazeo La Defense,4.7,ChIJx6pR16xl5kcRS4HSC_KGX7I,395,Eloïse N,14,True,I don't recommend this center. I had my last t...,...,model='gemma3:1b' created_at='2025-09-16T16:36...,gemma3:1b,2025-09-16T16:36:43.5584584Z,stop,unpredictable treatment,request detailed session notes and results ana...,329.0,0.0,NaN,NaN


In [15]:
final_merged_df.orig_index.nunique()

1828

In [18]:
final_merged_df.issue.value_counts()

issue
lack of long-term results                                                                        319
lack of follow-up                                                                                305
poor follow-up                                                                                   233
poor communication                                                                               180
unprofessional staff                                                                             177
poor customer service                                                                            174
poor reception / unfriendly welcome                                                              162
results not as promised / unmet expectations                                                     140
lack of professionalism                                                                          124
staff attitude                                                                       

In [20]:
final_merged_df

,orig_index,_row_index,title,totalScore,google_place_id,reviewsCount,name,reviewerNumberOfReviews,isLocalGuide,textTranslated,...,raw_output,model,created_at,done_reason,issue,actionable,source_row,json_block_idx,priority,quality_check
0,0,12,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Sarah C,29,True,I would like to share my experience after a ye...,...,model='gemma3:1b' created_at='2025-09-16T16:40...,gemma3:1b,2025-09-16T16:40:16.3469944Z,stop,hair regrowth complications,investigate machine programming,0.0,0.0,NaN,NaN
1,0,12,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Sarah C,29,True,I would like to share my experience after a ye...,...,model='gemma3:1b' created_at='2025-09-16T16:40...,gemma3:1b,2025-09-16T16:40:16.3469944Z,stop,poor treatment results,consult specialist,0.0,0.0,NaN,NaN
2,0,12,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Sarah C,29,True,I would like to share my experience after a ye...,...,model='gemma3:1b' created_at='2025-09-16T16:40...,gemma3:1b,2025-09-16T16:40:16.3469944Z,stop,lack of compensation for ineffective sessions,escalate complaint,0.0,0.0,NaN,NaN
3,0,12,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Sarah C,29,True,I would like to share my experience after a ye...,...,model='gemma3:1b' created_at='2025-09-16T16:40...,gemma3:1b,2025-09-16T16:40:16.3469944Z,stop,lack of doctor consultation,request consultation,0.0,0.0,NaN,NaN
4,0,12,Lazeo Rennes 2,4.8,ChIJ--Sssh3fDkgRwG7_On3ilEg,205,Sarah C,29,True,I would like to share my experience after a ye...,...,model='gemma3:1b' created_at='2025-09-16T16:40...,gemma3:1b,2025-09-16T16:40:16.3469944Z,stop,lack of clear diagnosis and treatment plan,request detailed explanation,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,329,29223,Lazeo La Defense,4.7,ChIJx6pR16xl5kcRS4HSC_KGX7I,395,Eloïse N,14,True,I don't recommend this center. I had my last t...,...,model='gemma3:1b' created_at='2025-09-16T16:36...,gemma3:1b,2025-09-16T16:36:43.5584584Z,stop,unclean glasses,inspect glasses and ensure disinfection,329.0,0.0,NaN,NaN
1342,329,29223,Lazeo La Defense,4.7,ChIJx6pR16xl5kcRS4HSC_KGX7I,395,Eloïse N,14,True,I don't recommend this center. I had my last t...,...,model='gemma3:1b' created_at='2025-09-16T16:36...,gemma3:1b,2025-09-16T16:36:43.5584584Z,stop,rushed sessions,review session duration and ensure adequate ti...,329.0,0.0,NaN,NaN
1343,329,29223,Lazeo La Defense,4.7,ChIJx6pR16xl5kcRS4HSC_KGX7I,395,Eloïse N,14,True,I don't recommend this center. I had my last t...,...,model='gemma3:1b' created_at='2025-09-16T16:36...,gemma3:1b,2025-09-16T16:36:43.5584584Z,stop,inconsistent results,request detailed session notes and results ana...,329.0,0.0,NaN,NaN
1344,329,29223,Lazeo La Defense,4.7,ChIJx6pR16xl5kcRS4HSC_KGX7I,395,Eloïse N,14,True,I don't recommend this center. I had my last t...,...,model='gemma3:1b' created_at='2025-09-16T16:36...,gemma3:1b,2025-09-16T16:36:43.5584584Z,stop,unpredictable treatment,request detailed session notes and results ana...,329.0,0.0,NaN,NaN


In [21]:
final_merged_df.columns

Index(['orig_index', '_row_index', 'title', 'totalScore', 'google_place_id',
       'reviewsCount', 'name', 'reviewerNumberOfReviews', 'isLocalGuide',
       'textTranslated', 'publishAt', 'publishedAtDate', 'visitedIn',
       'likesCount', 'stars', 'raw_output', 'model', 'created_at',
       'done_reason', 'issue', 'actionable', 'source_row', 'json_block_idx',
       'priority', 'quality_check'],
      dtype='object')

In [22]:
final_merged_df[['title','name', 'textTranslated', 'publishAt', 'publishedAtDate','visitedIn', 'stars','issue', 'actionable']].shape

(9093, 9)

In [34]:
final_merged_df2 = final_merged_df[['title','name', 'textTranslated', 'publishAt', 'publishedAtDate','visitedIn', 'stars','issue', 'actionable']].drop_duplicates(subset = ['title','name','issue'])

In [35]:
final_merged_df2.to_csv('negative_points_final.csv',index=0)

In [5]:
final_merged_df2

NameError: name 'final_merged_df2' is not defined

In [3]:
pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.7 MB 2.9 MB/s eta 0:00:03
   ------ --------------------------------- 1.3/8.7 MB 3.6 MB/s eta 0:00:03
   ------- -------------------------------- 1.6/8.7 MB 3.6 MB/s eta 0:00:02
   -------- ------------------------------- 1.8/8.7 MB 2.4 MB/s eta 0:00:03
   ------------ --------------------------- 2.6/8.7 MB 2.7 MB/s eta 0:00:03
   --------------- ------------------------ 3.4/8.7 MB 2.9 MB/s eta 0:00:02
   ------------------ --------------------- 3.9/8.7 MB 2.9 MB/s eta 0:00:02
   --------------------- ------------------ 4.7/8.7 MB 2.9 MB/s eta 0:00:02
   ------------------------ --------------- 5.2/8.7 MB 2.9 MB/s eta 0:00:02
   -------------------------- ------------- 5.8/8.7 MB 2.8 MB/s eta 0:00:02
   ---------------------------- ----------- 6.3/8.7 MB 2.8 MB/s eta 0:00:01
   --------------


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
neg_points = pd.read_csv('negative_points_final.csv')

In [27]:
neg_new = neg_points.groupby('issue')['title'].count().reset_index().sort_values(by='title',ascending = False)
neg_new['count'] = neg_new['title']
neg_new.to_csv('issue_temp.csv', index=0)

In [28]:
neg_new

,issue,title,count
1570,lack of long-term results,316,316
1507,lack of follow-up,303,303
2159,poor follow-up,233,233
2089,poor communication,179,179
2934,unprofessional staff,177,177
2121,poor customer service,173,173
2232,poor reception / unfriendly welcome,162,162
2473,results not as promised / unmet expectations,140,140
1631,lack of professionalism,122,122
2587,staff attitude,101,101


In [29]:
"""
Group similar issue strings into clusters and aggregate counts.

Input: CSV or DataFrame with columns: 'issue' and 'count' (counts numeric).
Output: DataFrame with cluster_id, representative_label, aggregated_count,
        member_issues (list), and optionally saved CSV.

Tweak:
 - distance_threshold: lower => stricter (fewer merges), higher => more merges.
"""

import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity

# ---------- Parameters ----------
INPUT_CSV = "issue_temp.csv"   # set to path or None if you pass df directly
OUTPUT_CSV = "issues_grouped.csv"
RAW_ISSUE_COL = "issue"   # name of the issue text column
COUNT_COL = "title"       # name of the count column (must be numeric)
DISTANCE_THRESHOLD = 0.35  # 0..1 where 0 merges only identical; ~0.25-0.45 is typical to try

# ---------- Preprocessing helpers ----------
def normalize_text(s: str) -> str:
    """Lowercase, strip, remove punctuation except internal &/&, collapse whitespace."""
    if not isinstance(s, str):
        s = str(s)
    s = s.lower().strip()
    # replace slashes and & with space to separate tokens
    s = s.replace("/", " ").replace("&", " ")
    # remove punctuation (keep alphanumerics and spaces)
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ---------- Main function ----------
def group_issues(df: pd.DataFrame,
                 issue_col: str = RAW_ISSUE_COL,
                 count_col: str = COUNT_COL,
                 distance_threshold: float = DISTANCE_THRESHOLD,
                 output_csv: str | None = OUTPUT_CSV) -> pd.DataFrame:
    # copy and validate
    df_in = df.copy()
    if issue_col not in df.columns or count_col not in df.columns:
        raise ValueError(f"DataFrame must contain columns '{issue_col}' and '{count_col}'")
    # ensure counts numeric
    df[count_col] = pd.to_numeric(df[count_col], errors="coerce").fillna(0).astype(int)

    # normalize text (we'll keep original issues too)
    df["issue_norm"] = df[issue_col].astype(str).map(normalize_text)

    # If there are many identical normalized strings, combine them first to reduce size
    combined = df.groupby("issue_norm", as_index=False).agg({
        issue_col: lambda x: x.iloc[0],    # keep one original representative
        count_col: "sum"
    }).rename(columns={issue_col: "original_issue"})

    texts = combined["issue_norm"].tolist()
    counts = combined[count_col].values

    # Compute TF-IDF vectors
    vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=1)
    X = vectorizer.fit_transform(texts)  # shape (n_texts, n_features)

    # Compute cosine distance matrix (1 - similarity)
    sim = cosine_similarity(X)
    # Guard against numerical issues
    sim = np.clip(sim, 0.0, 1.0)
    dist = 1.0 - sim

    # If only one unique text, just return it as single cluster
    n = len(texts)
    if n == 0:
        return pd.DataFrame()  # nothing to do
    if n == 1:
        out = pd.DataFrame([{
            "cluster_id": 0,
            "representative": combined["original_issue"].iloc[0],
            "agg_count": int(counts[0]),
            "members": [combined["original_issue"].iloc[0]],
            "member_norms": [texts[0]]
        }])
        if output_csv:
            out.to_csv(output_csv, index=False)
        return out

    # Agglomerative clustering on the precomputed distance matrix.
    # Use n_clusters=None and distance_threshold to allow automatic cluster count.
    clustering = AgglomerativeClustering(
        n_clusters=None,
        metric="precomputed",
        linkage="average",
        distance_threshold=distance_threshold
    )
    labels = clustering.fit_predict(dist)

    combined["cluster_id"] = labels

    # Build cluster aggregated table
    cluster_rows = []
    for cid, group in combined.groupby("cluster_id"):
        # pick representative: the member with largest original count
        # (you can change to shortest string, centroid, or manual label)
        rep_idx = group[count_col].idxmax()
        rep_label = group.loc[rep_idx, "original_issue"]
        agg_count = int(group[count_col].sum())
        members = group["original_issue"].tolist()
        member_norms = group["issue_norm"].tolist()
        cluster_rows.append({
            "cluster_id": int(cid),
            "representative": rep_label,
            "agg_count": agg_count,
            "members": members,
            "member_norms": member_norms,
            "member_counts": group[count_col].tolist()
        })

    out_df = pd.DataFrame(cluster_rows).sort_values("agg_count", ascending=False).reset_index(drop=True)

    if output_csv:
        out_df.to_csv(output_csv, index=False)

    return out_df

# ---------- Example usage ----------
if __name__ == "__main__":
    # Option 1: read from CSV
    try:
        df_in = pd.read_csv(INPUT_CSV)
    except FileNotFoundError:
        raise SystemExit(f"Input CSV not found: {INPUT_CSV}")
    # tolerate uppercase column names
    cols = {c.lower(): c for c in df_in.columns}
    issue_col = next((cols[k] for k in cols if k == RAW_ISSUE_COL.lower()), RAW_ISSUE_COL)
    count_col = next((cols[k] for k in cols if k == COUNT_COL.lower()), COUNT_COL)

    grouped = group_issues(df_in, issue_col=issue_col, count_col=count_col,
                           distance_threshold=DISTANCE_THRESHOLD, output_csv=OUTPUT_CSV)

    print("Done. Top clusters:")
    print(grouped.head(30).to_string(index=False))


Done. Top clusters:
 cluster_id                                 representative  agg_count                                                                                                                                                                                              members                                                                                                                                                                                 member_norms     member_counts
        205                              lack of follow-up        324                                                                    [Communication / Lack of Follow-up, lack of follow-up, lack of follow-up / communication, Poor Communication & Lack of Follow-Up]                                                                  [communication lack of follow up, lack of follow up, lack of follow up communication, poor communication lack of follow up]    [1, 316, 5, 2]
        179                     

In [19]:
neg_new

,issue,title
1570,lack of long-term results,316
1507,lack of follow-up,303
2159,poor follow-up,233
2089,poor communication,179
2934,unprofessional staff,177
...,...,...
6,4 different people,1
5,unsatisfactory results,1
4,unprofessional staff behavior,1
3,unprofessional conduct,1


In [22]:
grouped[['representative','agg_count']]

,representative,agg_count
0,lack of follow-up,324
1,lack of long-term results,318
2,unprofessional staff,263
3,poor customer service,260
4,poor follow-up,236
5,poor communication,192
6,poor reception / unfriendly welcome,173
7,results not as promised / unmet expectations,141
8,lack of professionalism,132
9,poor communication / unclear information,123


In [30]:
"""
Aggressive grouping of issue strings.

Two strategies:
 - GREEDY TOKEN-JACCARD MERGE (pure Python)  <-- recommended for aggressive merging
 - TFIDF + AGGLOMERATIVE (if scikit-learn is available)

Input CSV must contain columns: 'issue' and 'count' (case-insensitive).
"""

import math
import re
import json
import pandas as pd
from collections import Counter, defaultdict
from itertools import combinations
from typing import List, Set, Tuple, Dict

# ---------- PARAMETERS ----------
INPUT_CSV = "issue_temp.csv"      # input CSV path (issue, count)
OUTPUT_GREEDY = "issues_grouped_greedy.csv"
OUTPUT_TFIDF = "issues_grouped_tfidf.csv"

# Greedy merge threshold (token-set Jaccard). Lower => MORE aggressive merging.
MERGE_THRESHOLD = 0.30   # try 0.25 - 0.45; 0.30 is aggressive but not extreme

# TF-IDF agglomerative (only used if sklearn available)
USE_TFIDF_IF_AVAILABLE = True
TFIDF_DISTANCE_THRESHOLD = 0.5  # 0..1 where 1 merges everything; lower => stricter. Increase to be more aggressive.

# ---------- TEXT NORMALIZATION ----------
def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    s = s.lower().strip()
    # unify some tokens
    s = s.replace("/", " ").replace("&", " ")
    # remove punctuation except spaces
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def tokens_from_text(s: str) -> Set[str]:
    s = normalize_text(s)
    toks = set(t for t in s.split() if len(t) > 1)  # ignore 1-letter tokens
    return toks if toks else set(s.split())  # fallback to whatever

# ---------- GREEDY TOKEN-JACCARD MERGING ----------
def jaccard(a: Set[str], b: Set[str]) -> float:
    if not a or not b:
        return 0.0
    inter = len(a & b)
    uni = len(a | b)
    return inter / uni if uni else 0.0

def greedy_token_merge(df: pd.DataFrame, issue_col="issue", count_col="count", threshold=MERGE_THRESHOLD):
    # Prepare initial clusters: one per normalized issue (merge exact identical normals)
    df = df.copy()
    df[count_col] = pd.to_numeric(df[count_col], errors="coerce").fillna(0).astype(int)
    df["issue_norm"] = df[issue_col].astype(str).map(normalize_text)

    grouped = df.groupby("issue_norm", as_index=False).agg({
        issue_col: lambda s: s.iloc[0],
        count_col: "sum"
    }).rename(columns={issue_col: "rep_issue"})

    # cluster structure: id -> {members:set(original issues), count:int, tokenset:set}
    clusters = {}
    for i, row in grouped.iterrows():
        toks = tokens_from_text(row["issue_norm"])
        clusters[i] = {
            "members": [row["rep_issue"]],
            "count": int(row[count_col]),
            "tokenset": toks,
            "rep_issue": row["rep_issue"]
        }

    # Precompute pair similarities lazily; iterative merging
    def find_best_pair(clusters_dict):
        best = (None, None, -1.0)  # (id1, id2, score)
        ids = list(clusters_dict.keys())
        for a_i in range(len(ids)):
            for b_j in range(a_i+1, len(ids)):
                i = ids[a_i]; j = ids[b_j]
                score = jaccard(clusters_dict[i]["tokenset"], clusters_dict[j]["tokenset"])
                if score > best[2]:
                    best = (i, j, score)
        return best

    # Iteratively merge highest-similarity pair if >= threshold
    iteration = 0
    while True:
        iteration += 1
        if len(clusters) <= 1:
            break
        i, j, score = find_best_pair(clusters)
        if i is None or score < threshold:
            break
        # merge j into i (choose larger count as target to keep rep)
        if clusters[j]["count"] > clusters[i]["count"]:
            i, j = j, i
        # merge j -> i
        clusters[i]["members"].extend(clusters[j]["members"])
        clusters[i]["count"] += clusters[j]["count"]
        clusters[i]["tokenset"] = clusters[i]["tokenset"] | clusters[j]["tokenset"]
        # pick representative as highest-count member (we only keep original strings, approximate by length)
        # keep existing rep_issue (could refine)
        del clusters[j]
        # Continue loop (aggressive merging)
    # build output DataFrame
    rows = []
    for cid, c in clusters.items():
        # unique members and sort by count? we don't have per-member counts now — approximate
        uniq_members = list(dict.fromkeys(c["members"]))
        rows.append({
            "representative": c["rep_issue"],
            "agg_count": c["count"],
            "members": json.dumps(uniq_members, ensure_ascii=False)
        })
    out = pd.DataFrame(rows).sort_values("agg_count", ascending=False).reset_index(drop=True)
    return out

# ---------- TF-IDF + AGGLOMERATIVE (if sklearn available) ----------
def tfidf_agglomerative(df: pd.DataFrame, issue_col="issue", count_col="count", distance_threshold=TFIDF_DISTANCE_THRESHOLD):
    try:
        from sklearn.feature_extraction.text import TfidfVectorizer
        from sklearn.cluster import AgglomerativeClustering
        from sklearn.metrics.pairwise import cosine_similarity
        import sklearn
    except Exception as e:
        print("sklearn not available or failed to import:", e)
        return None

    df = df.copy()
    df[count_col] = pd.to_numeric(df[count_col], errors="coerce").fillna(0).astype(int)
    df["issue_norm"] = df[issue_col].astype(str).map(normalize_text)
    grouped = df.groupby("issue_norm", as_index=False).agg({
        issue_col: lambda s: s.iloc[0],
        count_col: "sum"
    }).rename(columns={issue_col: "rep_issue"})
    texts = grouped["issue_norm"].tolist()
    vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=1)
    X = vectorizer.fit_transform(texts)
    sim = cosine_similarity(X)
    dist = 1.0 - sim
    # compatibility for sklearn versions
    skl_ver = tuple(int(x) for x in getattr(__import__("sklearn"), "__version__").split(".")[:2])
    if skl_ver >= (1,2):
        clustering = AgglomerativeClustering(n_clusters=None, metric="precomputed", linkage="average", distance_threshold=distance_threshold)
    else:
        clustering = AgglomerativeClustering(n_clusters=None, affinity="precomputed", linkage="average", distance_threshold=distance_threshold)

    labels = clustering.fit_predict(dist)
    grouped["cluster_id"] = labels
    # aggregate per cluster
    rows = []
    for cid, g in grouped.groupby("cluster_id"):
        # rep = highest count
        rep_idx = g[count_col].idxmax()
        rep_label = g.loc[rep_idx, "rep_issue"]
        agg = int(g[count_col].sum())
        members = g["rep_issue"].tolist()
        rows.append({"cluster_id": int(cid), "representative": rep_label, "agg_count": agg, "members": json.dumps(members, ensure_ascii=False)})
    out = pd.DataFrame(rows).sort_values("agg_count", ascending=False).reset_index(drop=True)
    return out

# ---------- RUN ----------
if __name__ == "__main__":
    # load
    df = pd.read_csv(INPUT_CSV)
    # tolerate uppercase/lowercase column names
    cols = {c.lower(): c for c in df.columns}
    if "issue" in cols and "count" in cols:
        issue_col, count_col = cols["issue"], cols["count"]
    else:
        raise ValueError("Input CSV must contain columns 'issue' and 'count' (case-insensitive).")

    print("Running GREEDY token-Jaccard merging with MERGE_THRESHOLD =", MERGE_THRESHOLD)
    greedy_out = greedy_token_merge(df, issue_col=issue_col, count_col=count_col, threshold=MERGE_THRESHOLD)
    greedy_out.to_csv(OUTPUT_GREEDY, index=False)
    print("Greedy result saved to", OUTPUT_GREEDY)
    print(greedy_out.head(30).to_string(index=False))

    if USE_TFIDF_IF_AVAILABLE:
        print("\nAttempting TF-IDF + agglomerative clustering (if sklearn installed).")
        tfidf_out = tfidf_agglomerative(df, issue_col=issue_col, count_col=count_col, distance_threshold=TFIDF_DISTANCE_THRESHOLD)
        if tfidf_out is not None:
            tfidf_out.to_csv(OUTPUT_TFIDF, index=False)
            print("TF-IDF agglomerative result saved to", OUTPUT_TFIDF)
            print(tfidf_out.head(30).to_string(index=False))
        else:
            print("TF-IDF method skipped (sklearn not available).")

    print("\nDone.")


Running GREEDY token-Jaccard merging with MERGE_THRESHOLD = 0.3
Greedy result saved to issues_grouped_greedy.csv
                                representative  agg_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [33]:
tfidf_out.shape

(1942, 4)

In [34]:
greedy_out

,representative,agg_count,members
0,lack of follow-up,334,"[""lack of follow-up"", ""lack of customer follow..."
1,lack of long-term results,323,"[""lack of long-term results"", ""lack of noticea..."
2,poor follow-up,275,"[""poor follow-up"", ""Follow-up & Continuity"", ""..."
3,unprofessional staff behavior,271,"[""unprofessional staff behavior"", ""staff unpro..."
4,poor customer service,219,"[""poor customer service"", ""poor customer exper..."
5,results not as promised / unmet expectations,211,"[""results not as promised / unmet expectations..."
6,poor communication,205,"[""poor communication"", ""doctor communication"",..."
7,unprofessional staff,202,"[""unprofessional staff"", ""staff unprofessional..."
8,poor reception / unfriendly welcome,179,"[""poor reception / unfriendly welcome"", ""poor ..."
9,lack of compensation or goodwill gestures,154,"[""lack of compensation or goodwill gestures"", ..."


In [35]:
import pandas as pd
import json

# Paths
INITIAL_FILE = "issue_temp.csv"           # your original issues file
GREEDY_FILE = "issues_grouped_greedy.csv"    # output from greedy clustering
OUTPUT_FILE = "issues_with_greedy.csv"

# Load files
df_initial = pd.read_csv(INITIAL_FILE)
df_greedy = pd.read_csv(GREEDY_FILE)

# Expand the members column (which is a JSON array or comma-separated string)
def parse_members(m):
    if pd.isna(m):
        return []
    try:
        return json.loads(m) if isinstance(m, str) and m.strip().startswith("[") else [x.strip() for x in str(m).split(",")]
    except Exception:
        return [str(m)]

df_greedy["members_list"] = df_greedy["members"].apply(parse_members)

# Build mapping issue -> representative
issue_to_rep = {}
for _, row in df_greedy.iterrows():
    rep = row["representative"]
    for member in row["members_list"]:
        issue_to_rep[member] = rep

# Map each issue in initial file
df_initial["cluster_representative"] = df_initial["issue"].map(issue_to_rep)

# Save
df_initial.to_csv(OUTPUT_FILE, index=False)
print(f"Saved merged file with greedy cluster representatives: {OUTPUT_FILE}")
print(df_initial.head(20))


Saved merged file with greedy cluster representatives: issues_with_greedy.csv
                                           issue  title  count  \
0                      lack of long-term results    316    316   
1                              lack of follow-up    303    303   
2                                 poor follow-up    233    233   
3                             poor communication    179    179   
4                           unprofessional staff    177    177   
5                          poor customer service    173    173   
6            poor reception / unfriendly welcome    162    162   
7   results not as promised / unmet expectations    140    140   
8                        lack of professionalism    122    122   
9                                 staff attitude    101    101   
10                  slow or insufficient results     96     96   
11                               long wait times     93     93   
12                         lack of communication     89     89  

In [44]:
df_initial.groupby(['cluster_representative','issue'])['count'].count().reset_index().sort_values(by='count',ascending = 0)

,cluster_representative,issue,count
0,botched job,botched job,1
1,4 different people,4 different people,1
2,4 different people,unpleasant people,1
3,A strange experience. I've been going to Lazeo...,A strange experience. I've been going to Lazeo...,1
4,Appointment Cancellation,Appointment Cancellation,1
5,Appointment Cancellation,Appointment Cancellation Duration,1
6,Appointment Cancellation,Appointment Cancellation Frequency,1
7,Appointment Cancellation,appointment duration,1
8,Appointment Cancellation,doctor's appointment duration,1
9,Appointment Cancellation & Lack of Confirmation,Appointment Cancellation & Lack of Confirmation,1


In [48]:
neg_points

,title,name,textTranslated,publishAt,publishedAtDate,visitedIn,stars,issue,actionable
0,Lazeo Rennes 2,Sarah C,I would like to share my experience after a ye...,6 months ago,2025-02-24T08:14:41.537Z,Visited in February,1,hair regrowth complications,investigate machine programming
1,Lazeo Rennes 2,Sarah C,I would like to share my experience after a ye...,6 months ago,2025-02-24T08:14:41.537Z,Visited in February,1,poor treatment results,consult specialist
2,Lazeo Rennes 2,Sarah C,I would like to share my experience after a ye...,6 months ago,2025-02-24T08:14:41.537Z,Visited in February,1,lack of compensation for ineffective sessions,escalate complaint
3,Lazeo Rennes 2,Sarah C,I would like to share my experience after a ye...,6 months ago,2025-02-24T08:14:41.537Z,Visited in February,1,lack of doctor consultation,request consultation
4,Lazeo Rennes 2,Sarah C,I would like to share my experience after a ye...,6 months ago,2025-02-24T08:14:41.537Z,Visited in February,1,lack of clear diagnosis and treatment plan,request detailed explanation
...,...,...,...,...,...,...,...,...,...
8887,Lazeo La Defense,Eloïse N,I don't recommend this center. I had my last t...,2 years ago,2023-01-14T13:02:07.907Z,Visited in January 2023,2,unclean glasses,inspect glasses and ensure disinfection
8888,Lazeo La Defense,Eloïse N,I don't recommend this center. I had my last t...,2 years ago,2023-01-14T13:02:07.907Z,Visited in January 2023,2,rushed sessions,review session duration and ensure adequate ti...
8889,Lazeo La Defense,Eloïse N,I don't recommend this center. I had my last t...,2 years ago,2023-01-14T13:02:07.907Z,Visited in January 2023,2,inconsistent results,request detailed session notes and results ana...
8890,Lazeo La Defense,Eloïse N,I don't recommend this center. I had my last t...,2 years ago,2023-01-14T13:02:07.907Z,Visited in January 2023,2,unpredictable treatment,request detailed session notes and results ana...


In [53]:
neg_points2 = neg_points.merge(df_initial[['cluster_representative','issue']], on= 'issue')

In [59]:
neg_points2 = neg_points2.rename(columns={'issue':'old_issue'})

In [61]:
neg_points2['issue'] = np.where(neg_points2['cluster_representative'].isna(),neg_points2['old_issue'],neg_points2['cluster_representative'])

In [62]:
neg_points2[neg_points2['cluster_representative'].isna()]

,title,name,textTranslated,publishAt,publishedAtDate,visitedIn,stars,old_issue,actionable,cluster_representative,issue
29,Lazeo Val D'Europe,Patricia M,I'd like to provide some feedback to help pote...,2 months ago,2025-06-14T22:27:36.135Z,Visited in April,1,lack of empathy/consideration,improve customer care,NaN,lack of empathy/consideration
30,Lazeo Val D'Europe,Patricia M,I'd like to provide some feedback to help pote...,2 months ago,2025-06-14T22:27:36.135Z,Visited in April,1,poor hygiene/cleanliness,cleaning,NaN,poor hygiene/cleanliness
80,Lazeo Val D'Europe,Myriam Lavergne,No respect for schedules... when you make an a...,a year ago,2023-11-08T12:29:10.596Z,Visited in November 2023,1,Schedule Issues,improve scheduling,NaN,Schedule Issues
81,Lazeo Val D'Europe,Myriam Lavergne,No respect for schedules... when you make an a...,a year ago,2023-11-08T12:29:10.596Z,Visited in November 2023,1,Hygiene Issues,cleaning,NaN,Hygiene Issues
82,Lazeo Val D'Europe,Myriam Lavergne,No respect for schedules... when you make an a...,a year ago,2023-11-08T12:29:10.596Z,Visited in November 2023,1,Staff Negligence,staff training,NaN,Staff Negligence
...,...,...,...,...,...,...,...,...,...,...,...
8796,Lazeo Colmar,Dogan Dilek,"Left for hair-to-hair laser, when I arrived at...",a year ago,2023-10-03T20:31:41.864Z,Visited in March 2023,2,Poor communication,clarify expectations,NaN,Poor communication
8852,Lazeo La Defense,Lea Teixeira Novais,I carried out all of my laser hair removal ses...,4 months ago,2025-05-07T11:54:11.710Z,Visited in March,2,difficult booking/ unresponsive contact,improve booking process and communication chan...,NaN,difficult booking/ unresponsive contact
8856,Lazeo La Defense,Adèle Bao,Very mixed reviews...\n\nThe prices are obviou...,2 years ago,2023-06-03T21:34:44.987Z,Visited in June 2023,2,staff attitude / unprofessional behavior,staff training,NaN,staff attitude / unprofessional behavior
8871,Lazeo La Defense,Eloïse N,I don't recommend this center. I had my last t...,2 years ago,2023-01-14T13:02:07.907Z,Visited in January 2023,2,rushed sessions,review session duration and ensure adequate ti...,NaN,rushed sessions


In [63]:
neg_points2.to_csv('negative_points_final2.csv',index=0)

In [69]:
neg_points2[neg_points2['title'] == 'Lazeo Rennes 2'].nunique()

title                      1
name                       6
textTranslated             6
publishAt                  4
publishedAtDate            6
                          ..
stars                      2
old_issue                 20
actionable                13
cluster_representative    18
issue                     18
Length: 11, dtype: int64